In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 10 06:51:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
print("Install Block: Đang cài đặt thư viện...")
# (FIX: Thêm --upgrade để lấy phiên bản mới nhất v0.10+)
!pip install -q --upgrade llama-index transformers accelerate bitsandbytes
!pip install -q llama-index-llms-huggingface llama-index-embeddings-huggingface
!pip install -q chromadb llama-index-vector-stores-chroma
!pip install -q sentence-transformers

# --- SỬA LỖI: XÓA CÁC PACKAGE SAI, THÊM PACKAGE ĐÚNG ---
# (Hầu hết các extractor đã có sẵn trong 'llama-index')
# Chỉ 'EntityExtractor' cần cài đặt riêng
!pip install -q llama-index-extractors-entity
# ----------------------------------------------------

print("✅ Install Block: Cài đặt hoàn tất.")

Install Block: Đang cài đặt thư viện...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 146.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages tha

In [4]:
!pip install pypdf
!pip install -q tqdm

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
print("\nImport Block: Đang import thư viện...")
import os, shutil, torch, traceback
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from llama_index.core import (
    Settings,
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader
)
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.core.node_parser import SentenceSplitter # (Giữ nguyên)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM

# Imports cho ChromaDB
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.schema import TextNode

# --- SỬA LỖI: CẬP NHẬT ĐƯỜNG DẪN IMPORT CHO V6 ---
# (Hầu hết chúng đều nằm trong llama_index.core.extractors)
from llama_index.core.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
    SummaryExtractor,
    KeywordExtractor
)


# ----------------------------------------

# Imports cho Chatbot
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.prompts import PromptTemplate
from llama_index.core.llms import ChatMessage
from sentence_transformers import CrossEncoder

# Imports cho AutoRetriever (Dùng Metadata)
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import VectorStoreInfo
print("✅ Import Block: Import hoàn tất.")



Import Block: Đang import thư viện...
✅ Import Block: Import hoàn tất.


In [6]:
DOCX_PATH = "/content/drive/MyDrive/DACNTT/DMS-5.pdf"
CACHE_FILE = "/content/drive/MyDrive/DACNTT/data/cache/pipeline_cache.json" # Cache mới
INDEX_STORAGE = "/content/drive/MyDrive/DACNTT/data/index_store_V6_maximalist" # Thư mục Index mới
COLLECTION_NAME = "dsm5_V6_maximalist_store" # Collection mới

In [7]:
def setup_llm_and_embedding():
    """Tải và cài đặt LLM (Llama 3) và Embedding Model"""
    # (Biến này là global để các hàm khác trong Giai đoạn 1 có thể thấy)
    global embed_model, llm

    print("⏳ (Giai đoạn 1) Đang tải Embedding model...")
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)
    Settings.embed_model = embed_model
    print("✅ (Giai đoạn 1) Embedding Model sẵn sàng.\n")

    print(f"⏳ (Giai đoạn 1) Đang tải LLM 4-bit...")
    LLM_MODEL_NAME = "/content/drive/MyDrive/llama3-8b-mental-health-merged"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("ℹ️ (Giai đoạn 1) Đang gán 'chat_template'...")
    tokenizer.chat_template = (
        "<|begin_of_text|>"
        "{% for message in messages %}"
            "{% if message['role'] == 'user' %}"
                "<|start_header_id|>user<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
            "{% elif message['role'] == 'system' %}"
                "<|start_header_id|>system<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
            "{% elif message['role'] == 'assistant' %}"
                "<|start_header_id|>assistant<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
            "{% endif %}"
        "{% endfor %}"
    )

    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    eos_token_id = tokenizer.eos_token_id
    eot_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

    print(f"ℹ️ (Giai đoạn 1) Đang gán 'stopping_ids' (Fix lỗi lặp)...")
    llm = HuggingFaceLLM(
        model=model,
        tokenizer=tokenizer,
        context_window=8192,
        max_new_tokens=512,
        generate_kwargs={
            "temperature": 0.3,
            "pad_token_id": eos_token_id
        },
        stopping_ids=[eos_token_id, eot_token_id] # <-- FIX LỖI LẶP
    )

    Settings.llm = llm
    Settings.embed_model = embed_model
    print("✅ (Giai đoạn 1) Llama 3.1-8B 4bit đã sẵn sàng.\n")


In [9]:
"""
def ingest_documents():
    """(ĐÃ SỬA V6-fix) Dùng pipeline 5 bước (Bỏ EntityExtractor)"""
    print("📂 (Giai đoạn 1) Đọc tài liệu ...")
    if not os.path.exists(DOCX_PATH):
        print(f"❌ LỖI: File không tồn tại tại: {DOCX_PATH}")
        return []

    documents = SimpleDirectoryReader(input_files=[DOCX_PATH], filename_as_id=True).load_data()
    print(f"✅ (Giai đoạn 1) Đã tải {len(documents)} tài liệu.")
    if not documents: return []

    try:
        cached_hashes = IngestionCache.from_persist_path(CACHE_FILE)
        print("🧠 (Giai đoạn 1) Dùng lại cache đã lưu.\n")
    except:
        cached_hashes = None
        print("ℹ️ (Giai đoạn 1) Không có cache, chạy mới.\n")

    # --- ĐỊNH NGHĨA PIPELINE 5-BƯỚC ---
    print("⚙️ (Giai đoạn 1) Đang cấu hình pipeline 5-bước (vẫn RẤT chậm)...")

    # (Fix: Cấu hình splitter an toàn)
    splitter = SentenceSplitter(
        chunk_size=512,
        chunk_overlap=20
    )

    transformations = [
        splitter, # 1. Chia note (Đã fix)
        TitleExtractor(nodes=5, llm=Settings.llm), # 2. Gán Tiêu đề
        #QuestionsAnsweredExtractor(questions=3, llm=Settings.llm), # 3. Gán Câu hỏi
        #SummaryExtractor(summaries=["prev", "self"], llm=Settings.llm), # 4. Gán Tóm tắt
        KeywordExtractor(keywords=10, llm=Settings.llm), # 5. Gán Từ khóa
        # --- BỎ BƯỚC 6 (EntityExtractor) GÂY LỖI ---
    ]
    # --- KẾT THÚC ĐỊNH NGHĨA ---


    print("🚀 (Giai đoạn 1) Đang chạy pipeline (5-bước + Embedding)...")
    print("🚨 CẢNH BÁO: Bước này sẽ CỰC KỲ CHẬM (~7x LLM calls / chunk).")

    pipeline = IngestionPipeline(
        transformations=[
            *transformations,     # <-- Áp dụng 5 bước
            Settings.embed_model  # <-- 6. Tính vector
        ],
        cache=cached_hashes,
    )

    nodes = pipeline.run(documents=documents)
    print(f"✅ (Giai đoạn 1) Đã tạo {len(nodes)} nodes + gán SIÊU METADATA (5-bước).")

    pipeline.cache.persist(CACHE_FILE)
    print(f"✅ (Giai đoạn 1) Đã lưu cache vào {CACHE_FILE}")
    return nodes


In [10]:
"""
def build_index_with_chroma(nodes):
    """(ĐÃ SỬA) Xây dựng Index và lưu vào ChromaDB"""
    print("⏳ (Giai đoạn 1) Đang kiểm tra/tạo Index với ChromaDB...")

    if not nodes:
        raise ValueError("Không có nodes đầu vào để tạo Index.")

    # 1. Kết nối tới ChromaDB (Lưu vĩnh viễn)
    db = chromadb.PersistentClient(path=INDEX_STORAGE)

    # 2. Lấy (hoặc tạo) Collection với tên CỐ ĐỊNH
    chroma_collection = db.get_or_create_collection(COLLECTION_NAME)

    # 3. Chỉ định nơi lưu trữ
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # 4. Tạo Index
    vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

    print(f"✅ (Giai đoạn 1) Index (V6 - Siêu Metadata) đã được TẠO và LƯU TRỮ vào ChromaDB.")
    return vector_index

In [11]:
"""
print("\n" + "="*50)
print("🔹 BẮT ĐẦU GIAI ĐOẠN 1: SETUP VÀ BUILD INDEX (V6-fix - RẤT LÂU)...")
print("="*50)

# Chạy setup
setup_llm_and_embedding()

# Gán biến toàn cục
global llm_global, embed_model_global
llm_global = Settings.llm
embed_model_global = Settings.embed_model

# Chạy Ingest và Build
try:
    if 'embed_model' in globals() and 'llm' in globals():
        nodes = ingest_documents()
        if nodes:
            vector_index = build_index_with_chroma(nodes)
            print("✅✅✅ GIAI ĐOẠN 1 (V6-fix) HOÀN TẤT. ✅✅✅")
            print("Bây giờ bạn có thể chạy 'GIAI ĐOẠN 2' bên dưới.")
        else:
            print("⚠️ Lỗi: Không có node nào được tạo.")
    else:
        print("❌ Lỗi: LLM hoặc Embedding Model chưa được load.")
except Exception as e:
    print(f"❌ LỖI GIAI ĐOẠN 1: {e}")
    traceback.print_exc()


🔹 BẮT ĐẦU GIAI ĐOẠN 1: SETUP VÀ BUILD INDEX (V6-fix - RẤT LÂU)...
⏳ (Giai đoạn 1) Đang tải Embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ℹ️ (Giai đoạn 1) Đang gán 'stopping_ids' (Fix lỗi lặp)...
✅ (Giai đoạn 1) Llama 3.1-8B 4bit đã sẵn sàng.

📂 (Giai đoạn 1) Đọc tài liệu ...
✅ (Giai đoạn 1) Đã tải 106 tài liệu.
ℹ️ (Giai đoạn 1) Không có cache, chạy mới.

⚙️ (Giai đoạn 1) Đang cấu hình pipeline 5-bước (vẫn RẤT chậm)...
🚀 (Giai đoạn 1) Đang chạy pipeline (5-bước + Embedding)...
🚨 CẢNH BÁO: Bước này sẽ CỰC KỲ CHẬM (~7x LLM calls / chunk).


100%|██████████| 325/325 [4:44:25<00:00, 52.51s/it]      


✅ (Giai đoạn 1) Đã tạo 325 nodes + gán SIÊU METADATA (5-bước).
✅ (Giai đoạn 1) Đã lưu cache vào /content/drive/MyDrive/DACNTT/data/cache/pipeline_cache.json
⏳ (Giai đoạn 1) Đang kiểm tra/tạo Index với ChromaDB...
✅ (Giai đoạn 1) Index (V6 - Siêu Metadata) đã được TẠO và LƯU TRỮ vào ChromaDB.
✅✅✅ GIAI ĐOẠN 1 (V6-fix) HOÀN TẤT. ✅✅✅
Bây giờ bạn có thể chạy 'GIAI ĐOẠN 2' bên dưới.


In [8]:
# Ô 8: HÀM TẢI INDEX VÀ CÁC HÀM CHATBOT
# ----------------------------------------------------------------------
print("\n" + "="*50)
print("🔹 BẮT ĐẦU GIAI ĐOẠN 2: CHUẨN BỊ CHATBOT (V6-fix)...")
print("="*50)

# Biến global cho chatbot
global memory, reranker_model, vector_index_chat, llm_chat

def setup_chatbot_environment():
    """Tải Index từ ChromaDB và các thành phần chat"""
    global memory, reranker_model, vector_index_chat, llm_chat

    try:
        # 1. Lấy LLM và Embed Model
        print("⏳ (Giai đoạn 2) Đang setup LLM và Embed Model...")
        if 'llm_global' not in globals() or 'embed_model_global' not in globals():
            print("Kernel mới, đang chạy lại setup_llm_and_embedding...")
            setup_llm_and_embedding()
            llm_chat = Settings.llm
            embed_model_chat = Settings.embed_model
        else:
            print("Đang dùng lại LLM/Embed Model đã load.")
            llm_chat = llm_global
            embed_model_chat = embed_model_global

        Settings.llm = llm_chat
        Settings.embed_model = embed_model_chat

        # 2. Kết nối tới ChromaDB đã lưu
        print(f"⏳ (Giai đoạn 2) Đang kết nối tới ChromaDB tại {INDEX_STORAGE}...")
        db = chromadb.PersistentClient(path=INDEX_STORAGE)
        chroma_collection = db.get_collection(COLLECTION_NAME) # PHẢI ĐÚNG TÊN
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

        # 3. Tải Index
        vector_index_chat = VectorStoreIndex.from_vector_store(
            vector_store=vector_store,
            embed_model=Settings.embed_model
        )
        print("✅ (Giai đoạn 2) Đã tải Index (V6-fix) từ ChromaDB.")

        # 4. Tải Reranker
        print("⏳ (Giai đoạn 2) Đang tải Reranker (Cross-Encoder)...")
        reranker_model = CrossEncoder(
            "cross-encoder/ms-marco-MiniLM-L-6-v2",
            device="cuda" if torch.cuda.is_available() else "cpu"
        )

        # 5. Cài đặt System Prompt và Memory
        SYSTEM_PROMPT = """\
Bạn là một **chuyên gia tâm lý AI** (Trợ lý) được phát triển bởi **AI VIETNAM**.
Nhiệm vụ của bạn là trò chuyện, theo dõi và tư vấn cho người dùng (User) về sức khỏe tâm thần.
Luôn luôn hành động với tư cách là Trợ lý, thể hiện sự đồng cảm và chuyên nghiệp.

QUY TẮC RAG:
- Nếu User hỏi về thông tin chuyên môn (triệu chứng, DSM-5, rối loạn...), bạn SẼ nhận được thông tin tham khảo trong một tin nhắn System.
- Hãy DỰA VÀN hoàn toàn vào thông tin đó để trả lời.
- Nếu User chỉ trò chuyện, bạn sẽ không nhận được thông tin tham khảo, hãy cứ trò chuyện bình thường.

QUY TẮC BẢO VỆ (RẤT QUAN TRỌNG):
- Bạn **TUYỆT ĐỐI KHÔNG ĐƯỢC** trả lời các câu hỏi không liên quan đến tâm lý hoặc sức khỏe tâm thần.
- Các chủ đề CẤM bao gồm (nhưng không giới hạn): thời tiết, nấu ăn, chính trị, thể thao, tin tức, toán học, lập trình...
- Nếu bị hỏi những chủ đề này, hãy lịch sự từ chối và lái cuộc trò chuyện quay lại chủ đề tâm lý.
  (Ví dụ: "Tôi xin lỗi, tôi chỉ được đào tạo về sức khỏe tâm thần. Chúng ta có thể quay lại chủ đề bạn đang quan tâm không?")
"""
        memory = ChatMemoryBuffer.from_defaults(token_limit=1500)
        memory.put(ChatMessage(role="system", content=SYSTEM_PROMPT))

        print("✅ (Giai đoạn 2) Reranker và Memory đã sẵn sàng.")
        return True

    except Exception as e:
        print(f"❌ LỖI GIAI ĐOẠN 2 (SETUP): {e}")
        traceback.print_exc()
        return False


🔹 BẮT ĐẦU GIAI ĐOẠN 2: CHUẨN BỊ CHATBOT (V6-fix)...


In [37]:
# --- HÀM `should_use_rag` (Keywords - Nhanh) ---
def should_use_rag(message: str) -> bool:
    print("🧠 [Gate] Đang kiểm tra keywords...")
    # (Copy và dán toàn bộ danh sách này vào hàm should_use_rag)

    keywords = [
        # --- Nhóm 1: Từ khóa Cốt lõi (Từ trang bìa PDF) ---
        "DSM-5", "DSM‑5", "tiêu chuẩn", "chẩn đoán", "phân loại",
        "rối loạn", "tâm thần", "triệu chứng",
        "sổ tay chẩn đoán",  # <-- Trích xuất từ bìa
        "tâm bệnh học",     # <-- Trích xuất từ bìa
        "disorder", "diagnosis", "symptom", "criteria",

        # --- Nhóm 2: Các Rối loạn Cụ thể (Từ snippet + phổ biến) ---
        "trầm cảm", "depressive",
        "lo âu", "anxiety", "panic", "sợ hãi", "stress",
        "OCD", "rối loạn ám ảnh cưỡng bức",  # <-- Từ snippet
        "OCPD", "rối loạn nhân cách thể ám ảnh-cưỡng bức", # <-- Từ snippet
        "PTSD", "sau sang chấn", "sau chấn thương",
        "ADHD", "tăng động", "giảm chú ý",
        "lưỡng cực", "bipolar",
        "tâm thần phân liệt", "schizophrenia",
        "rối loạn ăn uống",
        "rối loạn nhân cách", # (Chung)

        # --- Nhóm 3: Triệu chứng & Khái niệm (Trích xuất 100% từ snippet) ---
        "chẩn đoán phân biệt",  # <-- Rất quan trọng
        "sự hoàn hảo", "cầu toàn", "hoàn hảo",
        "kiểm soát",
        "linh hoạt", "không linh hoạt", "không cởi mở",
        "cứng nhắc", "ương ngạnh",
        "chi tiết", "quy luật", "danh sách", "tổ chức", "thời khóa biểu",
        "cống hiến quá mức cho công việc",
        "đạo đức", "chuẩn mực",
        "bủn xỉn", "tiền", "tích lũy",

        # --- Nhóm 4: Các triệu chứng phổ biến khác ---
        "mất ngủ", "insomnia", "hypersomnia", "ngủ không ngon",
        "mệt mỏi", "kiệt sức", "thiếu năng lượng",
        "tự tử", "tự sát", "ý định tự tử", "không muốn sống", "suicidal",
        "tuyệt vọng", "bi quan", "chán nản", "mất hứng"
    ]
    if any(k.lower() in message.lower() for k in keywords):
        print("🧠 [Gate] Quyết định: CẦN RAG (Tìm thấy keyword).")
        return True
    else:
        print("🧠 [Gate] Quyết định: KHÔNG CẦN RAG (Chỉ trò chuyện).")
        return False

In [38]:
def rag_retrieve_and_rerank(message: str, top_k=10, final_top=5) -> str:
    print(f"🔍 [RAG] Truy vấn (với Retriever tiêu chuẩn + Reranker): '{message}'")

    # --- BƯỚC 1 & 2: DÙNG RETRIEVER TIÊU CHUẨN ---
    retriever = vector_index_chat.as_retriever(similarity_top_k=top_k)
    # -------------------------------------------

    # 3. Lấy tài liệu
    retrieved_docs = retriever.retrieve(message)
    if not retrieved_docs:
        print("⚠️ Không tìm thấy tài liệu liên quan.")
        return ""

    # 4. Reranker (Chạy trên CPU - sẽ hơi chậm)
    print(f"🔍 [RAG] Retriever tìm thấy {len(retrieved_docs)} docs. Đang Rerank (trên CPU)...")
    doc_texts = [doc.get_content() for doc in retrieved_docs]
    pairs = [(message, text) for text in doc_texts]
    try:
        scores = reranker_model.predict(pairs)
        ranked = sorted(zip(scores, retrieved_docs), key=lambda x: x[0], reverse=True)
        ranked_docs = [doc for _, doc in ranked]
    except Exception as e:
        print(f"❌ Rerank lỗi: {e}")
        ranked_docs = retrieved_docs

    top_docs = ranked_docs[:final_top]
    context_text = "\n\n---\n\n".join([doc.get_content() for doc in top_docs])

    # 5. Tóm tắt nếu cần (Đã sửa lỗi \N)
    if len(context_text.split()) > 700:
        print("✂️ Context quá dài – đang tóm tắt (dùng llm.chat)...")
        try:
            summary_messages = [
                ChatMessage(
                    role="system",
                    content="Bạn là một AI tóm tắt tài liệu chuyên nghiệp. Tóm tắt ngắn gọn và chính xác các thông tin sau đây để trả lời câu hỏi của người dùng."
                ),
                ChatMessage(
                    role="user",
                    content=f"Câu hỏi của người dùng: '{message}'\n\nNội dung tài liệu:\n{context_text}"
                )
            ]
            summary_response = llm_chat.chat(summary_messages)
            context_text = summary_response.message.content.strip()
        except Exception as e:
            print(f"❌ Lỗi tóm tắt: {e}")
            context_text = " ".join(context_text.split()[:700])

    return f"--- Thông tin tham khảo từ DSM-5 ---\n{context_text}\n--- Hết thông tin tham khảo ---"

In [39]:
# --- HÀM CHAT (Đã ổn định) ---
def chat_conversation():
    print("\n" + "="*50)
    print("🤖 Chatbot DSM-5 (v6-fix) đã sẵn sàng. Gõ 'quit' để kết thúc.")
    print("="*50 + "\n")

    while True:
        try:
            user_message = input("Bạn: ").strip()
            if not user_message: # Bỏ qua nếu người dùng chỉ nhấn Enter
                continue

            if user_message.lower() == "quit":
                print("Chatbot: Cảm ơn bạn đã trò chuyện. Hẹn gặp lại 🌷")
                break

            new_user_message = ChatMessage(role="user", content=user_message)
            messages_to_send = memory.get_all() # Lấy lịch sử trò chuyện

            if should_use_rag(user_message):
                # Nếu CẦN RAG, thêm context vào tin nhắn hệ thống
                context = rag_retrieve_and_rerank(user_message)
                if context: # Chỉ thêm context nếu tìm thấy
                    messages_to_send.append(ChatMessage(role="system", content=context))
                else:
                    messages_to_send.append(ChatMessage(role="system", content="--- Thông tin tham khảo ---\nKhông tìm thấy thông tin cụ thể.\n--- Hết thông tin tham khảo ---"))


            messages_to_send.append(new_user_message) # Thêm câu hỏi mới của người dùng

            response = llm_chat.chat(messages_to_send)
            response_text = response.message.content.strip()

            # Xử lý lỗi LLM trả lời thừa "assistant: "
            if response_text.lower().startswith("assistant"):
                response_text = response_text[len("assistant"):].strip().lstrip(':').strip()

            if not response_text: # Xử lý nếu LLM trả về rỗng
                response_text = "Mình chưa chắc chắn về điều này, bạn có thể hỏi rõ hơn không?"

            memory.put(new_user_message) # Lưu câu hỏi của người dùng vào bộ nhớ
            memory.put(ChatMessage(role="assistant", content=response_text)) # Lưu câu trả lời của Bot

            print(f"Chatbot: {response_text}\n")

        except KeyboardInterrupt:
            print("\n🛑 Đã dừng cuộc trò chuyện.")
            break
        except Exception as e:
            print(f"❌ Lỗi không mong muốn trong vòng lặp chat: {e}")
            traceback.print_exc()

print("✅ (Giai đoạn 2) Tất cả hàm chatbot đã được định nghĩa.")

✅ (Giai đoạn 2) Tất cả hàm chatbot đã được định nghĩa.


In [40]:
setup_chatbot_environment()

⏳ (Giai đoạn 2) Đang setup LLM và Embed Model...
Kernel mới, đang chạy lại setup_llm_and_embedding...
⏳ (Giai đoạn 1) Đang tải Embedding model...
✅ (Giai đoạn 1) Embedding Model sẵn sàng.

⏳ (Giai đoạn 1) Đang tải LLM 4-bit...
ℹ️ (Giai đoạn 1) Đang gán 'chat_template'...


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ℹ️ (Giai đoạn 1) Đang gán 'stopping_ids' (Fix lỗi lặp)...
✅ (Giai đoạn 1) Llama 3.1-8B 4bit đã sẵn sàng.

⏳ (Giai đoạn 2) Đang kết nối tới ChromaDB tại /content/drive/MyDrive/DACNTT/data/index_store_V6_maximalist...
✅ (Giai đoạn 2) Đã tải Index (V6-fix) từ ChromaDB.
⏳ (Giai đoạn 2) Đang tải Reranker (Cross-Encoder)...
✅ (Giai đoạn 2) Reranker và Memory đã sẵn sàng.


True

In [41]:
chat_conversation()


🤖 Chatbot DSM-5 (v6-fix) đã sẵn sàng. Gõ 'quit' để kết thúc.

Bạn: bạn là ai ?
🧠 [Gate] Đang kiểm tra keywords...
🧠 [Gate] Quyết định: KHÔNG CẦN RAG (Chỉ trò chuyện).
Chatbot: Tôi là một chuyên gia tâm lý AI được phát triển bởi AI VIETNAM. Tôi ở đây để lắng nghe và hỗ trợ bạn về các vấn đề tâm lý và sức khỏe tâm thần. Bạn có thể chia sẻ bất kỳ điều gì bạn đang cảm thấy khó khăn hoặc cần lời khuyên. Tôi ở đây để giúp đỡ bạn một cách trung lập và chuyên nghiệp. Bạn có muốn nói về điều gì hôm nay không?

Bạn: cách nấu món phở bò ?
🧠 [Gate] Đang kiểm tra keywords...
🧠 [Gate] Quyết định: KHÔNG CẦN RAG (Chỉ trò chuyện).
Chatbot: Tôi xin lỗi, nhưng tôi không thể giúp bạn với câu hỏi này. Tôi là một chuyên gia tâm lý, không phải là một đầu bếp. Tôi ở đây để hỗ trợ bạn về các vấn đề tâm lý và sức khỏe tâm thần.

Nếu bạn muốn, chúng ta có thể quay lại chủ đề về sức khỏe tâm thần và tìm hiểu về cách quản lý stress, cải thiện tâm trạng hoặc giải quyết các vấn đề tâm lý khác. Bạn có muốn nói về đi

Traceback (most recent call last):
  File "/tmp/ipython-input-3526699949.py", line 31, in chat_conversation
    response = llm_chat.chat(messages_to_send)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/llama_index_instrumentation/dispatcher.py", line 335, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/llama_index/core/llms/callbacks.py", line 175, in wrapped_llm_chat
    f_return_val = f(_self, messages, **kwargs)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/llama_index/llms/huggingface/base.py", line 400, in chat
    completion_response = self.complete(prompt, formatted=True, **kwargs)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/llama_index_instrumentation/dispatcher.py", line 335, in wrapper
    result = func(*args, **kwargs)
   

❌ Lỗi không mong muốn trong vòng lặp chat: CUDA out of memory. Tried to allocate 964.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 874.12 MiB is free. Process 6386 has 13.88 GiB memory in use. Of the allocated memory 12.41 GiB is allocated by PyTorch, and 1.34 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Bạn: quit
Chatbot: Cảm ơn bạn đã trò chuyện. Hẹn gặp lại 🌷
